In [ ]:
import os
import pandas as pd
import re

# Data source
# https://www.ecad.eu/indicesextremes/customquerytimeseriesplots.php?optionSelected=index&processtext1=Your+query+is+being+processed.+Please+wait...&countryselect=SWITZERLAND%7Cch&stationselect=All+stations%7C**&categoryselect=All+categories%7C**&indexselect=CWD%3A+Maximum+no+of+consecutive+wet+days+%28RR+%3E%3D+1+mm%29%5B1%5D%7CCWD&seasonselect=ANNUAL%7C0&processtext2=Your+query+is+being+processed.+Please+wait...
base_folder = "weather_data"


In [ ]:
# List for all files
all_data = []

# Pass on each index
for index_folder in os.listdir(base_folder):
    index_path = os.path.join(base_folder, index_folder)
    if os.path.isdir(index_path):
        for filename in os.listdir(index_path):
            if filename.endswith(".txt"):
                filepath = os.path.join(index_path, filename)

                # We read the first 20 lines
                with open(filepath, 'r', encoding='utf-8') as file:
                    header_lines = [next(file) for _ in range(20)]

                # Looking for a line with information about the station
                station_line = [line for line in header_lines if "This is the time series data" in line][0]

                # Extract the name of the station and the Station-ID by regular expression
                match = re.search(r"SWITZERLAND,\s*(.+?)\s*\(Station-ID:\s*(\d+)\)", station_line)
                if match:
                    station_name = match.group(1).strip()
                    station_id = match.group(2).strip()
                else:
                    station_name = "Unknown"
                    station_id = "Unknown"

                # Read the data themselves
                df = pd.read_csv(filepath, skiprows=20, names=["YEAR", "CALC", "MEAN"])
                
                # Filter bad values
                df = df[df["YEAR"] != -999999]
                
                # Add the Index, Station and Station ID columns
                df["INDEX"] = index_folder
                df["STATION_NAME"] = station_name
                df["STATION_ID"] = station_id

                all_data.append(df)



In [ ]:
# Combine all the data
final_df = pd.concat(all_data, ignore_index=True)

In [ ]:
final_df["YEAR"] = pd.to_numeric(final_df["YEAR"], errors="coerce")
final_df["CALC"] = pd.to_numeric(final_df["CALC"], errors="coerce")
final_df["MEAN"] = pd.to_numeric(final_df["MEAN"], errors="coerce")
final_df["STATION_ID"] = pd.to_numeric(final_df["STATION_ID"], errors="coerce")

# Delete rows with incorrect values ​​in YEAR column
final_df = final_df.dropna(subset=["YEAR"])

# Convert YEAR column into an int type (after NAN removal)
final_df["YEAR"] = final_df["YEAR"].astype(int)

# Filter data from 2000 to 2024 inclusive
final_df = final_df[(final_df["YEAR"] >= 2000) & (final_df["YEAR"] <= 2024)]


In [37]:
final_df["CALC"] = final_df["CALC"] / 100

In [38]:
final_df

,YEAR,CALC,MEAN,INDEX,STATION_NAME,STATION_ID
100,2000,-9999.99,-999999.0,SD,COL DU GRAND ST-BERNARD,1649
101,2001,-9999.99,-999999.0,SD,COL DU GRAND ST-BERNARD,1649
102,2002,-9999.99,-999999.0,SD,COL DU GRAND ST-BERNARD,1649
103,2003,-9999.99,-999999.0,SD,COL DU GRAND ST-BERNARD,1649
104,2004,-9999.99,-999999.0,SD,COL DU GRAND ST-BERNARD,1649
...,...,...,...,...,...,...
90042,2020,0.00,0.0,TR,LUZERN,1655
90043,2021,0.00,0.0,TR,LUZERN,1655
90044,2022,1.00,0.0,TR,LUZERN,1655
90045,2023,1.00,0.0,TR,LUZERN,1655


In [39]:
# Store in CSV
final_df.to_csv("parsed_weather_data.csv", index=False)

print("✅ Successfully saved parsed_weather_data.csv!")

✅ Successfully saved parsed_weather_data.csv!


| Index | Description | How to understand CALC | How to use it |
|:-----|:-------------|:------------------------|:--------------|
| **CW** | Cold and Wet Days (days with low temperatures and precipitation) | Number of cold and wet days per year  | Higher risk of transport delays during extreme weather days. |
| **CWD** | Consecutive Wet Days (longest series of wet days) | Length of the longest consecutive rainy period (in 1 days) | If delivery overlaps with long rainy periods, expect a delay. |
| **FD** | Frost Days (days with minimum temperature below 0°C) | Number of frost days per year  | Frost can cause road icing; important to model delays in winter months. |
| **FG6BFT** | Strong Wind Days (days with wind ≥ 6 Beaufort) | Number of very windy days per year  | Strong winds can disrupt truck and cargo transport, especially in mountains. |
| **FGCALM** | Calm Days (days with very low wind) | Number of calm days per year  | Calm weather reduces risk; no significant impact on delays. |
| **R10MM** | Heavy Rain Days (days with ≥ 10mm precipitation) | Number of heavy rain days per year  | Heavy rain can slow transportation or cause minor disruptions. |
| **R20MM** | Very Heavy Rain Days (days with ≥ 20mm precipitation) | Number of very heavy rain days per year  | Severe rain events may flood roads and cause major delays. |
| **SD** | Snow Depth (maximum snow depth measured in a year) | Maximum snow cover height in centimeters (in 1 cm) | High snow depth significantly blocks or delays transport. |
| **SD1** | Snow Days > 1cm (days with snow cover > 1 cm) | Number of days with snow exceeding 1 cm  | Even small snow coverage can cause transport slowdowns. |
| **SD5CM** | Snow Days > 5cm (days with snow cover > 5 cm) | Number of days with snow exceeding 5 cm  | High snow accumulation can require rerouting or cause delays. |
| **SD50CM** | Snow Days > 50cm (days with snow cover > 50 cm) | Number of days with snow exceeding 50 cm  | Extreme snow events; consider delivery as almost impossible during such conditions. |
| **TR** | Tropical Nights (nights with minimum temperature > 20°C) | Number of tropical nights per year  | High nighttime temperatures have minor influence; can affect equipment or driver fatigue. |
| **WW** | Warm and Wet Days (days with high temperature and rain) | Number of warm and rainy days per year  | May cause fog, slippery conditions, and moderate delays. |